In [1]:
import pandas as pd

df = pd.read_json("../data/News_Category_Dataset_v3.json", lines=True)

In [2]:
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


Tokenize the Text Using BERT’s Tokenizer

Unlike LSTMs (which use simple tokenization), BERT requires: 

1️⃣ WordPiece tokenization (handles unknown words by breaking them into subwords).

2️⃣ Special tokens: [CLS] (start of sentence), [SEP] (separator for multi-segment inputs).

3️⃣ Fixed-length token sequences (padded/truncated).

In [3]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU Device Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


PyTorch Version: 2.6.0+cu118
CUDA Available: True
GPU Device Count: 1
GPU Name: NVIDIA GeForce RTX 3070 Ti Laptop GPU


Prepare Inputs for BERT
BERT requires inputs in a specific format:

- `input_ids`: Tokenized word indices
- `attention_mask`: Marks which tokens are real vs. padding
- `token_type_ids`: (Optional, used for sentence-pair tasks)

In [4]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the headlines and extract input_ids
df['input_ids'] = df['headline'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# Generate attention masks (1 for actual tokens, 0 for padding)
import torch
MAX_LEN = 50  # Define a max sequence length

encoded_inputs = tokenizer(
    df['headline'].tolist(), 
    padding="max_length", 
    truncation=True, 
    max_length=MAX_LEN, 
    return_tensors="pt"
)

# Store the attention mask in DataFrame
df['attention_mask'] = encoded_inputs["attention_mask"].tolist()

# Display results
df[['headline', 'input_ids', 'attention_mask']].head()


,headline,input_ids,attention_mask
0,Over 4 Million Americans Roll Up Sleeves For O...,"[101, 2058, 1018, 2454, 4841, 4897, 2039, 1511...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"American Airlines Flyer Charged, Banned For Li...","[101, 2137, 7608, 23821, 5338, 1010, 7917, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,23 Of The Funniest Tweets About Cats And Dogs ...,"[101, 2603, 1997, 1996, 4569, 15580, 2102, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,The Funniest Tweets From Parents This Week (Se...,"[101, 1996, 4569, 15580, 2102, 1056, 28394, 32...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,"[101, 2450, 2040, 2170, 10558, 2006, 2304, 474...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [5]:
df[['headline', 'input_ids', 'attention_mask']].to_json("bert_preprocessed.json", orient="records", lines=True)

In [6]:
df

,link,headline,category,short_description,authors,date,input_ids,attention_mask
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,"[101, 2058, 1018, 2454, 4841, 4897, 2039, 1511...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"[101, 2137, 7608, 23821, 5338, 1010, 7917, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,"[101, 2603, 1997, 1996, 4569, 15580, 2102, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,"[101, 1996, 4569, 15580, 2102, 1056, 28394, 32...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,"[101, 2450, 2040, 2170, 10558, 2006, 2304, 474...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28,"[101, 11418, 5766, 15321, 16173, 2002, 7076, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28,"[101, 3814, 21146, 2527, 6873, 3567, 9860, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28,"[101, 7230, 2058, 11579, 1010, 9924, 2058, 143...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28,"[101, 28163, 2078, 3044, 4727, 1024, 18156, 15...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."


In [8]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("CUDA Enabled:", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.16.1
CUDA Enabled: True
GPU Available: []


2025-03-12 23:42:57.725444: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-12 23:42:57.813129: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load pretrained BERT model
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

print("BERT Loaded on:", device)


2025-03-12 23:41:37.711039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 23:41:39.176747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


BERT Loaded on: cuda


In [ ]:
from torch.utils.data import Dataset, DataLoader

class BertDataset(Dataset):
    def __init__(self, df):
        self.input_ids = torch.tensor(df['input_ids'].tolist(), dtype=torch.long)
        self.attention_mask = torch.tensor(df['attention_mask'].tolist(), dtype=torch.long)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# Split data into train and validation sets
train_size = int(0.8 * len(df))
train_df, val_df = df[:train_size], df[train_size:]

# Create PyTorch datasets
train_dataset = BertDataset(train_df)
val_dataset = BertDataset(val_df)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Data ready for training!")


In [ ]:
from transformers import BertForMaskedLM

# Load BERT with a language modeling head
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.to(device)

print("Fine-tuning model loaded!")


In [ ]:
from transformers import AdamW

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

print("Optimizer and loss function set!")


In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train BERT
trainer.train()


In [ ]:
import optuna

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_epochs", 2, 5)

    # Define new training arguments
    training_args = TrainingArguments(
        output_dir="./bert_tuned_model",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir="./logs"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train and evaluate
    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_loss"]

# Run hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("Best hyperparameters:", study.best_params)


In [ ]:
import os

output_dir = "./bert_deploy"
os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved for deployment!")
